In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# sectors=['agriculture_clipped', 'mortality_clipped', 'energy_clipped', 'labor_clipped', 'AMEL_clipped', "CAMEL_clipped"]
sectors=['CAMEL_clipped']
recipe='risk_aversion'
disc='euler_ramsey'
eta_rhos = {
    2.0 :0.0,
    # 1.016010255: 9.149608e-05,
    # 1.244459066: 0.00197263997,
    # 1.421158116: 0.00461878399,
    # 1.567899395: 0.00770271076,
}
# xlim=(0,20)
xlim=(-1,8)

In [ ]:
for sector in sectors:
    for eta,rho in eta_rhos.items():
        
        df_library = f"/mnt/CIL_integration/damage_function_library/damage_function_library_rff3_coastalv19/{sector}/"
        results = f"/mnt/CIL_integration/rff3_with_disc_factors/iter0-19/{sector}/2020/unmasked_None"
        
        sccs = xr.open_dataset(
            f"{results}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_sccs.nc4"
        ).sel(weitzman_parameter='0.5').uncollapsed_sccs

        cutoff=sccs.chunk({'simulation': -1}).quantile(0.0001, ['rff_sp', 'simulation'])

        sccs = sccs.where(sccs < cutoff, drop=True)

        rff_ids = sccs.rff_sp.values

        coefs = xr.open_dataset(
            f"{df_library}/{recipe}_{disc}_eta{eta}_rho{rho}_damage_function_coefficients.nc4"
        ).sel(rff_sp=rff_ids)

        temps = xr.DataArray(
            np.arange(xlim[0],xlim[1], 0.25),
            coords={
                'anomaly' : np.arange(xlim[0], xlim[1], 0.25)
            })

        fit_rff = coefs['anomaly'] * temps + coefs['np.power(anomaly, 2)'] * temps ** 2
        fit_rff = fit_rff.to_dataframe('fit_rff').reset_index()

        g = sns.relplot(
            data=fit_rff,
                 x='anomaly',
                 y='fit_rff',
                 hue='year',
                 col='rff_sp',
                 col_wrap=3,
                 kind='line',
                 facet_kws={'sharey': False, 'sharex': True}
                )

        for i in range(len(rff_ids)):
            g.axes.flatten()[i].set_title(f'''rff_sp:{rff_ids[i]}
            {np.round(sccs.sel(rff_sp=rff_ids[i]).to_dataframe().uncollapsed_sccs.unique(),2)[1:]}'''
                                         )
        plt.subplots_adjust(hspace=0.2, top=0.9)
        g.fig.suptitle(f"Damage functions for RFF SPs in 0.01 percentile of SCCs \n {sector} {recipe} {disc}, eta={eta} rho={rho}")

        plt.savefig(
            f'/mnt/CIL_integration/rff_diagnostics/v3/damage_functions_negative_SCCs/{sector}_{recipe}_{disc}_{eta}_{rho}_xlim{xlim}.png', 
            bbox_inches='tight', 
            dpi=300
        )

In [ ]:
rff_ids